# <center> Service Titan Data Science Internship 2025</center>

## <center> Round 1 <center>

## <center> Aram Abrahamyan <center>

In [286]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import pickle
warnings.filterwarnings("ignore")

In [303]:
class DataExtractor:
    def __init__(self, path):
        self.path = path
        self.df = None
        self.data = None
        
    def load_data(self):
        try:
            self.data = pd.read_pickle(self.path)
            self.data_dirty = pd.DataFrame(self.data)
            print("Data loaded")
        except FileNotFoundError:
            print("File not found")
    
    def create_data_frame(self):
        if self.data_dirty is not None:
            customer_id = self.data_dirty.id
            customer_name = self.data_dirty.name
            registration_date = self.data_dirty.registration_date
            num_orders = self.data_dirty.orders.apply(lambda x: len(x) if isinstance(x, list) else 0)
            df_exp = self.data_dirty.explode('orders').reset_index(drop=True) #found from stackoverflow
            orders = pd.json_normalize(df_exp['orders'])
            self.df = pd.DataFrame({'customer_id': customer_id,
                                    'customer_name': customer_name,
                                    'registration_date': registration_date,
                                    'num_orders' : num_orders,
                                    'order_id' : order_id,
                                    'is_vip' : pd.Series([])
                                    })
            self.df = self.df.loc[self.df.index.repeat(self.df['num_orders'])].reset_index(drop=True) 
            self.df['order_id'] = orders['order_id']
            self.df['order_date'] = orders['order_date']
            self.df['items'] = orders['items']
            df_exp = self.df.explode('items').reset_index(drop=True)
            items = pd.json_normalize(df_exp['items'])
            self.df = pd.concat([df_exp.drop(columns=['items']), items_details], axis=1)
            self.df = self.df.rename(columns={'item_id': 'product_id',
                                              'price': 'unit_price',
                                              'quantity': 'item_quantity'})
            # Taken from stackoverflow to handel dollar sign and other non-numeric characters
            self.df['unit_price'] = (self.df['unit_price'].astype(str)
            .str.replace(r'[^0-9.\-]', '', regex=True)
            .replace('', pd.NA))
            self.df['unit_price'] = pd.to_numeric(self.df['unit_price'], errors='coerce')
            self.df['item_quantity'] = pd.to_numeric(self.df['item_quantity'], errors='coerce')
            self.df['total_item_price'] = self.df['unit_price'] * self.df['item_quantity']
            self.df['order_total'] = self.df.groupby('order_id')['total_item_price'].sum().reset_index(drop=True)
            self.df['total_order_value_percentage'] = (self.df['total_item_price'] / self.df['order_total']) * 100 # this one is quite ambigous, I don't know if I got it correct
            self.df.drop(columns=['num_orders', 'order_total'], inplace=True)    
            self.df = self.df.sort_values(by=['customer_id', 'order_id', 'product_id'], ascending=True).reset_index(drop=True)
            print("Data frame created")
        else:
            print("No data to create DataFrame")
    
    def add_vip_column(self):
        if self.df is not None:
            self.df['is_vip'] = np.where(self.df['customer_id'].isin([35, 11, 1 , 17, 23, 43, 4, 15, 8, 37]), True, False)
            print("VIP column added")
        else:
            print("No DataFrame to add VIP column")

    def assign_data_types(self):
        if self.df is not None:
            self.df['customer_id'] = self.df['customer_id'].astype('int64')
            self.df['customer_name'] = self.df['customer_name'].astype('string')
            self.df['registration_date'] = pd.to_datetime(self.df['registration_date'], errors='coerce') #to handle unparsable values
            #if the registratime exact time is not mentioned, it assumes that it is 00:00:00, in my opinnion it is better than just dropping them(for specific tasks we can review this part)
            #drop all rows with non-integer order_id
            self.df['order_id'] = pd.to_numeric(self.df['order_id'], errors='coerce')
            self.df['order_id'] = self.df['order_id'].astype('Int64')
            self.df['order_date'] = pd.to_datetime(self.df['order_date'], errors='coerce')
            self.df['product_id'] = self.df['product_id'].astype('Int64')
            self.df['product_name'] = self.df['product_name'].astype('string')
            self.df['category'] = self.df['category'].astype('string')
            self.df['unit_price'] = self.df['unit_price'].astype('float64')
            self.df['item_quantity'] = self.df['item_quantity'].astype('Int64')
            self.df['is_vip'] = self.df['is_vip'].astype('bool')
            print("Data types assigned")
        else:
            print("No DataFrame to assign data types")

    def map_categories(self):
        category_map = {1: 'Electronics',
                        2: 'Apparel',
                        3: 'Books',
                        4: 'Home Goods'
                        }
        self.df['category'] = self.df['category'].map(category_map).fillna('Misc')
    
            

In [304]:
data_extract = DataExtractor('customer_orders.pkl')
data_extract.load_data()
data_extract.create_data_frame()
data_extract.map_categories()
data_extract.add_vip_column()
data_extract.assign_data_types()
data_extract.df.head(40)

Data loaded
Data frame created
VIP column added
Data types assigned


,customer_id,customer_name,registration_date,order_id,is_vip,order_date,product_id,product_name,category,unit_price,item_quantity,total_item_price,total_order_value_percentage
0,1,Customer 1,2022-12-31 04:19:19,3,True,2023-09-06 00:42:50,1,Item 1 for Order 3,Electronics,377.96,1,377.96,84.560485
1,1,Customer 1,2022-12-31 04:19:19,84,True,2024-02-27 21:24:16,<NA>,<NA>,Misc,NaN,<NA>,NaN,NaN
2,1,Customer 1,2022-12-31 04:19:19,<NA>,True,2025-05-03 08:09:20,1,Item 1 for Order 2,Home Goods,52.70,2,105.40,4.694104
3,1,Customer 1,2022-12-31 04:19:19,<NA>,True,2025-05-03 08:09:20,2,Item 2 for Order 2,Misc,498.45,5,2492.25,121.364188
4,1,Customer 1,2022-12-31 04:19:19,<NA>,True,2025-05-03 08:09:20,3,Item 3 for Order 2,Misc,168.15,1,168.15,6.471689
5,2,Customer 2,2022-05-27 00:23:28,3,False,2023-01-01 07:32:50,<NA>,<NA>,Misc,NaN,<NA>,NaN,NaN
6,2,Customer 2,2022-05-27 00:23:28,5,False,2024-08-28 19:37:56,1,Item 1 for Order 5,Misc,342.68,2,685.36,33.166218
7,2,Customer 2,2022-05-27 00:23:28,5,False,2024-08-28 19:37:56,2,Item 2 for Order 5,Apparel,134.09,<NA>,NaN,NaN
8,2,Customer 2,2022-05-27 00:23:28,5,False,2024-08-28 19:37:56,3,Item 3 for Order 5,Electronics,295.97,1,295.97,84.739600
9,2,Customer 2,2022-05-27 00:23:28,5,False,2024-08-28 19:37:56,4,Item 4 for Order 5,Misc,316.01,4,1264.04,223.557709


In [302]:
data_extract.df.dtypes

customer_id                              int64
customer_name                           string
registration_date               datetime64[ns]
order_id                                 Int64
is_vip                                    bool
order_date                      datetime64[ns]
product_id                               Int64
product_name                            string
category                                string
unit_price                             float64
item_quantity                            Int64
total_item_price                       float64
total_order_value_percentage           float64
dtype: object

Some parts of the tasks were a bit ambigous, I have mantiond it as a comment. Some parts are taken from online sources, I have cited them in order to avoid inconviniences both from your and my sides(but I think there was no need to cite it, but just in case).

I was thinking of what to do with the missing values, so I decided to leave them as they are, because they are always wery task-specific that is why I have not dropped them. To further improve my task I could write more functions to handle missing values when converting the data type, but because of time preassure, I could not do that but it is not a big deal i think in this case, That one also is very task-specific.

In [305]:
data_extract.df.to_csv('output.csv', index=False)